In [1]:
import pandas as pd
import numpy as np

In [666]:
df = pd.read_csv('/home/michel/Documentos/Projetos_Portifolio/dataflow_apacheBeam/data.csv')
df.head(20)

,Título,Nome da Rua,Número,Bairro,Cidade,Preço,Área,Quartos,Banheiros,Garagens
0,Residencial bellas,Rua João Francisco dos Santos Pioneiros,NaN,NaN,Balneário Camboriú,R$ 2.490.000,137,3 Quartos,4 Banheiros,2 Garagens
1,Apartamento mobiliado,Rua Egidio Alfredo Crispim Pioneiros,NaN,NaN,Balneário Camboriú,R$ 1.690.000 Cond.: R$ 520,115,3 Quartos,2 Banheiros,4 Garagens
2,Apartamento em,Avenida Atlântica,NaN,4520 Centro,Balneário Camboriú,R$ 3.300.000 Cond.: R$ 1.000,140,3 Quartos,4 Banheiros,1 Garagem
3,Apartamento,Avenida Atlântica - Até 549/550,NaN,500 Barra Norte,Balneário Camboriú,R$ 18.900.000 Cond.: R$ 1.980,301,5 Quartos,7 Banheiros,4 Garagens
4,Residencial majestic,Rua 4450,NaN,64 Centro,Balneário Camboriú,R$ 6.450.000,257,3 Quartos,4 Banheiros,4 Garagens
5,Apartamento,Rua 902 Centro,NaN,NaN,Balneário Camboriú,R$ 1.550.000,121,3 Quartos,3 Banheiros,2 Garagens
6,Apartamento 3,Avenida Brasil,NaN,30 Centro,Balneário Camboriú,R$ 2.990.000 Cond.: R$ 2.129,122,3 Quartos,5 Banheiros,3 Garagens
7,Porto di,Endereço não disponível Centro,NaN,NaN,Balneário Camboriú,R$ 550.000,NaN,NaN,NaN,NaN
8,Apartamento mobiliado,Endereço não disponível Centro,NaN,NaN,Balneário Camboriú,R$ 2.000.000 Cond.: R$ 743,250,3 Quartos,2 Banheiros,3 Garagens
9,Sala no,Endereço não disponível Pioneiros,NaN,NaN,Balneário Camboriú,R$ 1.228.442,NaN,NaN,NaN,NaN


In [667]:
# Aplicar a lógica usando lambda e apply
df['Número'], df['Bairro'] = zip(*df.apply(lambda row: (row['Bairro'].split()[0], ' '.join(row['Bairro'].split()[1:])) if isinstance(row['Bairro'], str) and len(row['Bairro'].split()) > 1 else (row['Número'], row['Bairro']), axis=1))

# Converter 'Número' para int64
df['Número'] = pd.to_numeric(df['Número'], errors='coerce').astype('Int64')

In [668]:
df.insert(6, 'Taxa Cond', df.apply(lambda row: row['Preço'].split()[4] if isinstance(row['Preço'], str) and len(row['Preço'].split()) > 3 else None, axis=1))
df['Preço'] = df.apply(lambda row: row['Preço'].split()[1] if isinstance(row['Preço'], str) and len(row['Preço'].split()) > 2 else row['Preço'], axis=1)


In [669]:
df['Preço'] = df['Preço'].str.replace('R$ ', '')
df['Preço'] = df['Preço'].apply(lambda x: x[:-4] if x else '0')
df['Preço'] = df['Preço'].str.replace('.', '')

In [670]:
df['Preço'] = df['Preço'].apply(lambda x: float(x.replace('R$', '').replace('.', '').replace(',', '.')) if str(x).replace('.', '').isdigit() else np.nan)


In [671]:
df['Taxa Cond'] = df['Taxa Cond'].str.replace('.', '')
df['Taxa Cond'] = df['Taxa Cond'].replace('Cond.: R\$ ', '', regex=True).astype(float)
df.insert(7, 'Total Mensal', df.apply(lambda row: row['Preço'] + row['Taxa Cond'], axis=1))


In [672]:
df['Quartos'] = df['Quartos'].astype('str').apply(lambda x: x.split()[0])
df['Banheiros'] = df['Banheiros'].astype('str').apply(lambda x: x.split()[0])
df['Garagens'] = df['Garagens'].astype('str').apply(lambda x: x.split()[0])

In [673]:
df['Quartos'] = df['Quartos'].apply(lambda x: int(x) if str(x).isnumeric() else 0)
df['Banheiros'] = df['Banheiros'].apply(lambda x: int(x) if str(x).isnumeric() else 0)
df['Garagens'] = df['Garagens'].apply(lambda x: int(x) if str(x).isnumeric() else 0)

In [674]:
df['Área'] = df['Área'].apply(lambda x: float(x) if str(x).isnumeric() else 0)
df['Quartos'] = df['Quartos'].astype('Int64')
df['Banheiros'] = df['Banheiros'].astype('Int64')
df['Garagens'] = df['Garagens'].astype('Int64')

In [675]:
df.head(20)

,Título,Nome da Rua,Número,Bairro,Cidade,Preço,Taxa Cond,Total Mensal,Área,Quartos,Banheiros,Garagens
0,Residencial bellas,Rua João Francisco dos Santos Pioneiros,<NA>,NaN,Balneário Camboriú,2490.0,NaN,NaN,137.0,3,4,2
1,Apartamento mobiliado,Rua Egidio Alfredo Crispim Pioneiros,<NA>,NaN,Balneário Camboriú,1690.0,520.0,2210.0,115.0,3,2,4
2,Apartamento em,Avenida Atlântica,4520,Centro,Balneário Camboriú,3300.0,1000.0,4300.0,140.0,3,4,1
3,Apartamento,Avenida Atlântica - Até 549/550,500,Barra Norte,Balneário Camboriú,18900.0,1980.0,20880.0,301.0,5,7,4
4,Residencial majestic,Rua 4450,64,Centro,Balneário Camboriú,6450.0,NaN,NaN,257.0,3,4,4
5,Apartamento,Rua 902 Centro,<NA>,NaN,Balneário Camboriú,1550.0,NaN,NaN,121.0,3,3,2
6,Apartamento 3,Avenida Brasil,30,Centro,Balneário Camboriú,2990.0,2129.0,5119.0,122.0,3,5,3
7,Porto di,Endereço não disponível Centro,<NA>,NaN,Balneário Camboriú,550.0,NaN,NaN,0.0,0,0,0
8,Apartamento mobiliado,Endereço não disponível Centro,<NA>,NaN,Balneário Camboriú,2000.0,743.0,2743.0,250.0,3,2,3
9,Sala no,Endereço não disponível Pioneiros,<NA>,NaN,Balneário Camboriú,1228.0,NaN,NaN,0.0,0,0,0


In [676]:
df.dtypes

Título           object
Nome da Rua      object
Número            Int64
Bairro           object
Cidade           object
Preço           float64
Taxa Cond       float64
Total Mensal    float64
Área            float64
Quartos           Int64
Banheiros         Int64
Garagens          Int64
dtype: object

In [4]:
ver = pd.read_csv('/home/michel/Documentos/Projetos_Portifolio/dataflow_apacheBeam/data.csv')
ver.tail()

,Título,Nome da Rua,Número,Bairro,Cidade,Preço,Área,Quartos,Banheiros,Garagens
726,Apartamento,Avenida do Estado Dalmo Vieira,NaN,2495 Centro,Balneário Camboriú,R$ 1.190.000 Cond.: R$ 50,76,3 Quartos,2 Banheiros,2 Garagens
727,Casa,Rua Belmiro Diogo Cordeiro,NaN,-- Nova Esperança,Balneário Camboriú,R$ 430.000 Cond.: R$ 100,98,2 Quartos,1 Banheiro,1 Garagem
728,Casa mobiliada,Endereço não disponível Praia dos Amores,NaN,NaN,Balneário Camboriú,R$ 2.250.000,200,3 Quartos,3 Banheiros,1 Garagem
729,Apartamento,Rua Isidoro Caetano,NaN,190 Pioneiros,Balneário Camboriú,R$ 2.600.000 Cond.: R$ 1.100,163,3 Quartos,4 Banheiros,3 Garagens
730,Apartamento,Atlântica,NaN,471 Barra Norte,Balneário Camboriú,R$ 12.500.000,242,5 Quartos,5 Banheiros,4 Garagens
